## Minesweeper

We'll make a DataFrame that contains the necessary data for a game of Minesweeper: coordinates of the squares, whether the square contains a mine and the number of mines found on adjacent squares.

Let's suppose we're playing Minesweeper on a 5 by 4 grid:
```
X = 10
Y = 8
```
We'll populate a dataframe with the coordinates like this:
```
   x  y
0  0  0
1  0  1
2  0  2
3  0  3
...
```

In [1]:
import pandas as pd
import numpy as np

In [2]:
X = 10
Y = 8

x_array = np.arange(0, X)
y_array = np.arange(0, Y)

# For x repeat each value Y times
# For y repeat the array X times
coordinates = {"x": x_array.repeat(Y), "y": np.tile(y_array, X)}

df = pd.DataFrame(coordinates)
df

,x,y
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
75,9,3
76,9,4
77,9,5
78,9,6


We'll create a new column of zeros (safe) and ones (mine) and use a probability of a mine occuring at each location of `0.4`.

In [3]:
df["mine"] = np.random.binomial(1, 0.4, X * Y)
df

,x,y,mine
0,0,0,1
1,0,1,0
2,0,2,0
3,0,3,0
4,0,4,1
...,...,...,...
75,9,3,0
76,9,4,0
77,9,5,0
78,9,6,1


We'll create a new column called `'adjacent'`. This column should contain the number of mines found on adjacent squares in the grid. 

In [4]:
# Move df by a vector in all 8 dimensions and merge each time with original df
df["adjacent"] = (
    df.merge(df + [1, 1, 0], on=["x", "y"], suffixes=("_x1", "_y1"), how="left")
    .merge(df + [1, -1, 0], on=["x", "y"], suffixes=("_x2", "_y2"), how="left")
    .merge(df + [-1, 1, 0], on=["x", "y"], suffixes=("_x3", "_y3"), how="left")
    .merge(df + [-1, -1, 0], on=["x", "y"], suffixes=("_x4", "_y4"), how="left")
    .merge(df + [1, 0, 0], on=["x", "y"], suffixes=("_x5", "_y5"), how="left")
    .merge(df + [-1, 0, 0], on=["x", "y"], suffixes=("_x6", "_y6"), how="left")
    .merge(df + [0, 1, 0], on=["x", "y"], suffixes=("_x7", "_y7"), how="left")
    .merge(df + [0, -1, 0], on=["x", "y"], suffixes=("_x8", "_y8"), how="left")
    .iloc[:, 3:]
    .sum(axis=1)
)
df["adjacent"] = df["adjacent"].astype("int32")
df

,x,y,mine,adjacent
0,0,0,1,2
1,0,1,0,3
2,0,2,0,2
3,0,3,0,2
4,0,4,1,2
...,...,...,...,...
75,9,3,0,2
76,9,4,0,0
77,9,5,0,1
78,9,6,1,1


For each coordinates pair, we'll update the `adjacent` value to `NaN` if there is a mine in that position.

In [5]:
df.loc[df["mine"] == 1, "adjacent"] = np.nan
df

,x,y,mine,adjacent
0,0,0,1,NaN
1,0,1,0,3.0
2,0,2,0,2.0
3,0,3,0,2.0
4,0,4,1,NaN
...,...,...,...,...
75,9,3,0,2.0
76,9,4,0,0.0
77,9,5,0,1.0
78,9,6,1,NaN


Finally, we'll display the DataFrame in the form of a grid.

In [6]:
df.pivot_table(values="adjacent", index="y", columns="x")

x,0,1,2,3,4,5,6,7,8,9
y,,,,,,,,,,
0,NaN,NaN,3.0,2.0,NaN,2.0,1.0,0.0,1.0,NaN
1,3.0,NaN,NaN,3.0,3.0,NaN,1.0,1.0,3.0,3.0
2,2.0,3.0,4.0,NaN,4.0,4.0,3.0,2.0,NaN,NaN
3,2.0,NaN,4.0,5.0,NaN,NaN,NaN,2.0,2.0,2.0
4,NaN,5.0,NaN,NaN,NaN,NaN,4.0,2.0,0.0,0.0
5,NaN,5.0,NaN,NaN,NaN,5.0,NaN,1.0,1.0,1.0
6,2.0,5.0,NaN,6.0,4.0,NaN,2.0,1.0,2.0,NaN
7,NaN,3.0,NaN,NaN,2.0,1.0,1.0,0.0,2.0,NaN
